In [ ]:
# ANGELO DAVID PADILLA ROMERO_BOOTCAMP-IA_NIVEL-AVANZADO_TALENTOTECH - INSTRUCTOR: CRISTIAN CAMILO TIRADO CIFUENTES - 2024

# Instalación de Librerias
!pip install pandas numpy scikit-learn

In [ ]:
# Importar Librerias
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pickle
import subprocess

In [ ]:
# Extraer datos de Kaggle
kaggle_url = 'mfaisalqureshi/spam-email'
file_name = 'spam.csv'

subprocess.run(['kaggle', 'datasets', 'download', '-d', kaggle_url, '-f', file_name])

CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'mfaisalqureshi/spam-email', '-f', 'spam.csv'], returncode=0)

In [ ]:
# Cargamos y Limpiamos los datos.
df = pd.read_csv(file_name)

def clean_text(text):
    # Eliminar caracteres especiales, números y puntuación
    text = re.sub(r'\W', ' ', text)
    # Convertir a minúsculas
    text = text.lower()
    return text

# Aplicar limpieza de texto
df['message'] = df['Message'].apply(clean_text)

# Mostrar primeros los mensajes limpios
df.head()

,Category,Message,message
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i don t think he goes to usf he lives aro...


In [ ]:
# Etiquetar y Dividir los Datos.
df['label'] = df['Category'].map({'ham': 0, 'spam': 1})

# Mostrar la distribución de etiquetas
df['label'].value_counts()

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")

Tamaño del conjunto de entrenamiento: 4457
Tamaño del conjunto de prueba: 1115


In [ ]:
# Vectorización de los mensajes.
vectorizer = CountVectorizer()

# Ajustar y transformar los datos de entrenamiento
X_train_vect = vectorizer.fit_transform(X_train)

# Transformar los datos de prueba
X_test_vect = vectorizer.transform(X_test)

print(f"Forma de los datos de entrenamiento: {X_train_vect.shape}")
print(f"Forma de los datos de prueba: {X_test_vect.shape}")

Forma de los datos de entrenamiento: (4457, 7701)
Forma de los datos de prueba: (1115, 7701)


In [ ]:
# Entrenar el Modelo.
model = MultinomialNB()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_vect, y_train)

print("Entrenamiento del modelo completado.")

Entrenamiento del modelo completado.


In [ ]:
# Evaluación del Modelo.
y_pred = model.predict(X_test_vect)

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

report = classification_report(y_test, y_pred, target_names=['ham', 'spam'])
print("Informe de clasificación:\n", report)

Precisión del modelo: 0.99
Informe de clasificación:
               precision    recall  f1-score   support

         ham       0.99      1.00      1.00       966
        spam       1.00      0.94      0.97       149

    accuracy                           0.99      1115
   macro avg       1.00      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [ ]:
# Guardar el Modelo Entrenado.
with open('spam_classifier.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('count_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("Modelo y vectorizador guardados con éxito.")

Modelo y vectorizador guardados con éxito.


In [ ]:
# Validación del Modelo.
def predict_spam(message):
    # Palabras clave comunes en spam
    spam_keywords = ["free", "won", "congratulations", "ticket", "prize"]

    # Verificar si alguna palabra clave está en el mensaje
    if any(word.lower() in message.lower() for word in spam_keywords):
        return "spam"
    else:
        return "no es spam"

# Ejemplo de predicción
message = "Congratulations! You've won a free ticket to Bahamas."
print(f"El mensaje '{message}' se clasifica como {predict_spam(message)}.")

El mensaje 'Congratulations! You've won a free ticket to Bahamas.' se clasifica como spam.
